# Simpsons Character Classifier 

<img src="pic_0003.jpg" style="width:700px;height:400px;">

## Goals:
### 1. Reuse VGG 16  model to train on dog breed data
### 2. Freeze most of the layers and fine tune few top layers
### 3. Cut top layer and add new Fully Cunnected layer
### 4. Retrain just top layers 
### 5. EValuate Model


### Import python packages

In [ ]:


from keras import regularizers
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation,Dense
from keras.models import Sequential,load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3
from keras.applications import vgg16
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from numpy.random import rand
from keras.callbacks import ModelCheckpoint
import os
# Scikit Imports
from sklearn.model_selection import train_test_split
# Matplot Imports
import matplotlib.pyplot as plt
from keras.models import model_from_json

import seaborn as sns 


In [ ]:
# Intialize matplotlib parameters

pd.options.display.max_colwidth = 600


params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

plt.rcParams.update(params)
%matplotlib inline

# pandas display data frames as tables
from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

## Get Dataset

The Simposons available at [Kaggle/Dogbreed](https://www.kaggle.com/alexattia/the-simpsons-characters-dataset).


In [ ]:
train_folder = '/Volumes/My Passport for Mac/data/simpsons/'
test_folder = '/Volumes/My Passport for Mac/data/dog-breeds/test/'

In [ ]:
def getImagesFilePathsFromFolder(path):
    files = []
    print("path: " + path )
    for f in listdir(path):
        if( isfile(join(path, f)) and (".jpg" in f)) :
            #print("path: " + path)
            #print("file:" + f)
            #print("path + file", os.path.join(path, f))
            files.append(os.path.join(path, f))  
    return files

### Build dataset of labels and image paths

In [ ]:
data_dict = {}
target_labels_list = []
image_path_list = []
counter = 0
fileCounter = 0

walkIterations = 0
for r,d,f in os.walk(train_folder):
    print("Walk Iteration : " + str(walkIterations) )
    print("root : " + r)
    print(" dir: " + str(d))
    print("file: " + str(f))
    if(walkIterations == 0 ):
        walkIterations = walkIterations + 1
        continue
    fileList = getImagesFilePathsFromFolder(r)
    for file in fileList:
        label = file.split("/")[-2]
        image_path = file
        print("file : " + file)
        print("label : " + label)
        print("image_path : " + image_path)
        target_labels_list.append(label)
        image_path_list.append(image_path)
        counter = counter + 1
        fileCounter = fileCounter + 1
    walkIterations = walkIterations + 1
    
                
print(counter)            
data_dict["label"] = target_labels_list            
data_dict["image_path"] = image_path_list
input_df = pd.DataFrame(data_dict)


In [ ]:
input_df.head()

In [ ]:
input_df.describe()

## Check Number of Classes in the Dataset


In [ ]:
target_labels = input_df['label']
len(set(target_labels))
print(target_labels[0:5])

## Prepare Labels
Deep Learning models work with one hot encoded outputs or target variables. We utilize pandas to prepare one hot encoding for the labels.

In [ ]:
labels_ohe_names = pd.get_dummies(target_labels, sparse=True)

print(labels_ohe_names.shape)
labels_ohe = np.asarray(labels_ohe_names)
labels_ohe_names.to_csv("labels.csv")
print(type(labels_ohe_names))
print(labels_ohe.shape)
print(labels_ohe[0:5])
labels_ohe_names.head()

### Understand Data


In [ ]:
sns.set(style="darkgrid")
sns.set(color_codes=True)
fig, ax = plt.subplots()
fig.set_size_inches(17.7, 25.27)
sns.catplot(y="label",   kind="count", palette="ch:.25",ax=ax, data=input_df, order=input_df.label.value_counts().iloc[:20].index);


## Prepare Train-Test Datasets
We use a 70-30 split to prepare the two dataset. 

In [ ]:
train_data = np.array([img_to_array(
                            load_img(img, 
                                     target_size=(299, 299))
                       ) for img 
                           in input_df['image_path'].values.tolist()
                      ]).astype('float32')

In [ ]:
train_data.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_data, 
                                                    target_labels, 
                                                    test_size=0.3, 
                                                    stratify=np.array(target_labels), 
                                                    random_state=42)

In [ ]:
x_train.shape, x_test.shape

Prepare Validation Dataset

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                    y_train, 
                                                    test_size=0.15, 
                                                    stratify=np.array(y_train), 
                                                    random_state=42)

In [ ]:
x_train.shape, x_val.shape

In [ ]:
print(y_train.shape)

Prepare target variables for train, test and validation datasets

In [ ]:
y_train_ohe = pd.get_dummies(y_train.reset_index(drop=True)).as_matrix()
y_val_ohe = pd.get_dummies(y_val.reset_index(drop=True)).as_matrix()
y_test_ohe = pd.get_dummies(y_test.reset_index(drop=True)).as_matrix()

y_train_ohe.shape, y_test_ohe.shape, y_val_ohe.shape

## Data Augmentation

Since number of samples per class are not very high, we utilize data augmentation to prepare different variations of different samples available. We do this using the ```ImageDataGenerator utility``` from ```keras```

In [ ]:
BATCH_SIZE = 32

In [ ]:
# Create train generator.
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')
train_generator = train_datagen.flow(x_train, y_train_ohe, shuffle=False, batch_size=BATCH_SIZE, seed=1)

In [ ]:
# Create validation generator
val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow(x_val, y_val_ohe, shuffle=False, batch_size=BATCH_SIZE, seed=1)

## Prepare Deep Learning Classifier

* Load InceptionV3 pretrained on ImageNet without its top/classification layer
* Add additional custom layers on top of InceptionV3 to prepare custom classifier

In [ ]:
# Get the InceptionV3 model so we can do transfer learning
#base_inception = InceptionV3(weights='imagenet', include_top = False, input_shape=(299, 299, 3))
base_vgg16_model = vgg16.VGG16(weights='imagenet', include_top = False, input_shape=(299, 299, 3))

#print(base_inception.summary())

base_vgg16_model.trainable = True
set_trainable = False
for layer in base_vgg16_model.layers:
    print(layer)
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        print(layer.name)
        print("set trainable as true")
        layer.trainable = True
    else:
        print(layer.name)
        layer.trainable = False
        print("set trainable as false")



In [ ]:
# Add a global spatial average pooling layer
out = base_vgg16_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
total_classes = y_train_ohe.shape[1]
predictions = Dense(total_classes, activation='softmax')(out)

* Stack the two models (InceptionV3 and custom layers) on top of each other 
* Compile the model and view its summary

In [ ]:
model = Model(inputs=base_vgg16_model.input, outputs=predictions)

# only if we want to freeze layers
#for layer in base_inception.layers:
#    layer.trainable = False
    
model.summary()
# Compile 
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 


## Model Training
We train the model with a Batch Size of 32 for just 15 Epochs.

The model utilizes the power of transfer learning to achieve a validation accuracy of about __81%__ !

In [ ]:
# Train the model
batch_size = BATCH_SIZE
train_steps_per_epoch = x_train.shape[0] // batch_size
val_steps_per_epoch = x_val.shape[0] // batch_size
checkpoint = ModelCheckpoint('model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs=15,
                              callbacks=[checkpoint],
                              verbose=True)

## Visualize Model Performance

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
t = f.suptitle('Deep Neural Net Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epochs = list(range(1,16))
ax1.plot(epochs, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(epochs)
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epochs, history.history['loss'], label='Train Loss')
ax2.plot(epochs, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(epochs)
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

## Test Model Performance

Step 1 is to prepare the training dataset. Since we scaled training data, test data should also be scaled in a similar manner. 

_Note: Deep Learning models are very sensitive to scaling._

In [ ]:
# scaling test features
x_test /= 255.

In [ ]:
test_predictions = model.predict(x_test)
test_predictions

In [ ]:
predictions = pd.DataFrame(test_predictions, columns=labels_ohe_names.columns)
predictions.head()

In [ ]:
def get_x_test(img_path, img_height, img_width):
    return np.array([img_to_array(load_img(img_path, target_size=(img_height, img_width)))]).astype('float32')


In [ ]:
def predict(x_test, model):
    #x_test = kimage.resize(image, 299, 299) 
    x_test /= 255.0
    test_predictions = model.predict(x_test)
    predictions = pd.DataFrame(test_predictions, columns=labels_ohe_names.columns)
    predictions = list(predictions.idxmax(axis=1))
    return predictions
    

In [ ]:
def getImagesFilePathsFromFolder(path):
    onlyfiles = [ join(path,f) for f in listdir(path) if ( isfile(join(path, f)) and (".jpg" in f) )]
    return onlyfiles

In [ ]:
test_labels = list(y_test)
predictions = list(predictions.idxmax(axis=1))
predictions[:10]

### Save Model to disk

In [ ]:
MODEL_JSON_PATH = "/Volumes/My Passport for Mac/model/simpsons.json"
MODEL_H5_PATH = "/Volumes/My Passport for Mac/model/simpsons.h5"


# serialize model to JSON
model_json = model.to_json()
with open(MODEL_JSON_PATH, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_H5_PATH)
print("Saved model to disk")

### Load model from disk

In [ ]:
# load json and create model
json_file = open(MODEL_JSON_PATH, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(MODEL_H5_PATH)
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 


### Load test images and test predictions

In [ ]:
path = "/Volumes/My Passport for Mac/data/simpsons-test"
fileCount = len(getImagesFilePathsFromFolder(path)) 
#print(getImagesFilePathsFromFolder(path))
fig, ax = plt.subplots(1,fileCount, figsize=(50,50))
img_Counter=0;

for img_path in getImagesFilePathsFromFolder(path):
    print(img_path)
    breed = predict(get_x_test(img_path, 299, 299), loaded_model)[0]
    print(breed)
    ax[img_Counter].set_title(breed)
    ax[img_Counter].imshow(load_img(img_path, target_size=(299, 299)))
    img_Counter = img_Counter + 1



## Analyze Test Performance

In [ ]:
import model_evaluation_utils as meu

In [ ]:
meu.get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)

In [ ]:
meu.display_classification_report(true_labels=test_labels, 
                                  predicted_labels=predictions, 
                                  classes=list(labels_ohe_names.columns))

In [ ]:
meu.display_confusion_matrix(true_labels=test_labels, 
                                    predicted_labels=predictions, 
                                    classes=list(labels_ohe_names.columns))

The model achieves a test accuracy of approximately __86%__

## Visualize Model Performance
Visualize model performance with actual images, labels and prediction confidence

In [ ]:
grid_width = 5
grid_height = 5
f, ax = plt.subplots(grid_width, grid_height)
f.set_size_inches(15, 15)
batch_size = 25
dataset = x_test

label_dict = dict(enumerate(labels_ohe_names.columns.values))
model_input_shape = (1,)+model.get_input_shape_at(0)[1:]
random_batch_indx = np.random.permutation(np.arange(0,len(dataset)))[:batch_size]

img_idx = 0
for i in range(0, grid_width):
    for j in range(0, grid_height):
        actual_label = np.array(y_test)[random_batch_indx[img_idx]]
        prediction = model.predict(dataset[random_batch_indx[img_idx]].reshape(model_input_shape))[0]
        label_idx = np.argmax(prediction)
        predicted_label = label_dict.get(label_idx)
        conf = round(prediction[label_idx], 2)
        ax[i][j].axis('off')
        ax[i][j].set_title('Actual: '+actual_label+'\nPred: '+predicted_label + '\nConf: ' +str(conf))
        ax[i][j].imshow(dataset[random_batch_indx[img_idx]])
        img_idx += 1

plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.5, hspace=0.55)    